In [1]:
!pip install langchain langgraph cassio

In [2]:
!pip install langchain_community

In [3]:
!pip install -U tiktoken langchain-groq langchainhub chromadb langchain langgraph langchain_huggingface

In [ ]:
#Connecting Vector DB
import cassio


import os 
from dotenv import load_dotenv

load_dotenv()
## connection of the ASTRA DB
ASTRA_DB_APPLICATION_TOKEN= os.getenv("ASTRA_DB_APPLICATION_TOKEN")# enter the "AstraCS:..." string found in in your Token JSON file"
ASTRA_DB_ID= os.getenv("ASTRA_DB_ID")
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)


In [4]:
from langchain_community.vectorstores import chroma


    * will use chroma db insted of aster db.
    `step 1: urls = [], load and split the data`

    `step 2: create embeddings using huggingface model`

    `step 3: store embeddings in vector DB`

    ```step 4: from langchain.indexes.vectorstore import VectorStoreIndexWrapper
                astra_vector_store.add_documents(doc_splits)
                print("Inserted %i headlines." % len(doc_splits))

                astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)```

    `step 5: create a retriver`


urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",

    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [5]:
#Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]


#Load 

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

#split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap = 0
)

doc_splits = text_splitter.split_documents(docs_list)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
import os
from langchain_huggingface import HuggingFaceEmbeddings

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\pkhobragade\Desktop\Langgraph\Langgraph\langg\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\pkhobragade\Desktop\Langgraph\Langgraph\langg\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pkhobragade\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either 

In [10]:
from langchain_community.vectorstores import Chroma


vector_store = Chroma(
    collection_name="langgraph_demo",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

C:\Users\pkhobragade\AppData\Local\Temp\ipykernel_31196\1521201812.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(
Python-dotenv could not parse statement starting at line 1


In [11]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
vector_store.add_documents(doc_splits)
print("Inserted %i headlines." % len(doc_splits))

Inserted 88 headlines.
